In [16]:
import torch

DATA_PATH = "../csi_dataset/csi_windows_w64_s32.pt"  # <-- ตรงนี้เปลี่ยนเป็น path ของคุณ

obj = torch.load(DATA_PATH)
X_all = obj["X"]   # shape: (N, 32, 53)
y_all = obj["y"]   # shape: (N,)

print("X_all shape:", X_all.shape)
print("y_all shape:", y_all.shape)

X_all shape: torch.Size([173, 32, 53])
y_all shape: torch.Size([173])


In [17]:
from sklearn.model_selection import train_test_split
import numpy as np

# 1) แยก 15% ออกมาเป็น test set
X_temp, X_test, y_temp, y_test = train_test_split(
    X_all,
    y_all,
    test_size=0.15,       # 15% สำหรับ test
    random_state=42,      # fix seed เพื่อ reproducible
    stratify=y_all        # ให้สัดส่วน class ใน test ใกล้เคียงทั้ง dataset
)

print("Total N :", len(X_all))
print("Temp  N :", len(X_temp))  # 85%
print("Test  N :", len(X_test))  # 15%

# ดู distribution ของ class ก่อนและหลังแบ่ง
def show_class_dist(name, y):
    y_np = y.cpu().numpy() if isinstance(y, torch.Tensor) else np.array(y)
    uniq, cnt = np.unique(y_np, return_counts=True)
    print(f"{name} class distribution:")
    for u, c in zip(uniq, cnt):
        print(f"  class {u}: {c}")
    print()

show_class_dist("ALL ", y_all)
show_class_dist("TEMP", y_temp)
show_class_dist("TEST", y_test)

Total N : 173
Temp  N : 147
Test  N : 26
ALL  class distribution:
  class 0: 21
  class 1: 79
  class 2: 73

TEMP class distribution:
  class 0: 18
  class 1: 67
  class 2: 62

TEST class distribution:
  class 0: 3
  class 1: 12
  class 2: 11



In [18]:
from sklearn.model_selection import StratifiedKFold
import numpy as np
import torch

# แปลง label ให้เป็น numpy เพื่อใช้กับ StratifiedKFold
y_temp_np = y_temp.cpu().numpy() if isinstance(y_temp, torch.Tensor) else np.array(y_temp)

# กำหนดจำนวน fold
K = 5
kfold = StratifiedKFold(n_splits=K, shuffle=True, random_state=42)

In [19]:
input_size = X_temp.shape[2]    # น่าจะเป็น 53
num_classes = 3
EPOCHS = 20
BATCH_SIZE = 16
LR = 1e-3

fold_val_accs = []   # เอาไว้เก็บ val accuracy ของแต่ละ fold

In [20]:
import torch

DEVICE = torch.device(
    "mps" if torch.backends.mps.is_available() else
    "cuda" if torch.cuda.is_available() else
    "cpu"
)

print("Using device:", DEVICE)

Using device: mps


In [21]:
import torch
from torch.utils.data import Dataset

class CSIDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [22]:
import torch
import torch.nn as nn

class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMClassifier, self).__init__()

        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True
        )

        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # x shape: (batch_size, seq_len, input_size)

        out, _ = self.lstm(x)              # (B, seq_len, hidden)
        out = out[:, -1, :]                # เอา timestep สุดท้าย
        out = self.fc(out)                 # (B, num_classes)

        return out

In [23]:
from torch.utils.data import DataLoader, Dataset

fold_idx = 0

for train_idx, val_idx in kfold.split(np.zeros(len(y_temp_np)), y_temp_np):
    print(f"\n===== Fold {fold_idx} =====")
    
    # เลือก subset จาก X_temp, y_temp ตาม index ของ fold นี้
    X_train_fold = X_temp[train_idx]
    y_train_fold = y_temp[train_idx]
    X_val_fold   = X_temp[val_idx]
    y_val_fold   = y_temp[val_idx]

    print("  Train fold size:", len(X_train_fold))
    print("  Val   fold size:", len(X_val_fold))

    # สร้าง Dataset / DataLoader
    train_ds = CSIDataset(X_train_fold, y_train_fold)
    val_ds   = CSIDataset(X_val_fold,   y_val_fold)

    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False)

    # สร้าง model ใหม่สำหรับ fold นี้
    model = LSTMClassifier(
        input_size=input_size,
        hidden_size=64,
        num_layers=1,
        num_classes=num_classes
    ).to(DEVICE)

    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)

    # ===== Train บน fold นี้ =====
    for epoch in range(1, EPOCHS + 1):
        model.train()
        correct = 0
        total = 0

        for xb, yb in train_loader:
            xb = xb.to(DEVICE)
            yb = yb.to(DEVICE)

            optimizer.zero_grad()
            logits = model(xb)
            loss = criterion(logits, yb)
            loss.backward()
            optimizer.step()

            preds = logits.argmax(dim=1)
            correct += (preds == yb).sum().item()
            total += yb.size(0)

        train_acc = correct / total

        # วัด val ทุก epoch (หรือจะวัดเฉพาะ epoch สุดท้ายก็ได้)
        model.eval()
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for xb, yb in val_loader:
                xb = xb.to(DEVICE)
                logits = model(xb)
                preds = logits.argmax(dim=1).cpu()
                val_correct += (preds == yb).sum().item()
                val_total += yb.size(0)

        val_acc = val_correct / val_total

        print(f"  Fold {fold_idx} Epoch {epoch:02d} | train_acc={train_acc:.3f} | val_acc={val_acc:.3f}")

    # สมมติใช้ val_acc ของ epoch สุดท้ายเป็นตัวแทนของ fold นี้ก่อน (แบบง่าย)
    fold_val_accs.append(val_acc)

    fold_idx += 1

print("\nValidation accuracy ต่อ fold:", fold_val_accs)
print("Mean val accuracy over folds:", np.mean(fold_val_accs))


===== Fold 0 =====
  Train fold size: 117
  Val   fold size: 30
  Fold 0 Epoch 01 | train_acc=0.786 | val_acc=0.833
  Fold 0 Epoch 02 | train_acc=0.889 | val_acc=0.867
  Fold 0 Epoch 03 | train_acc=0.855 | val_acc=0.867
  Fold 0 Epoch 04 | train_acc=0.940 | val_acc=0.967
  Fold 0 Epoch 05 | train_acc=1.000 | val_acc=0.967
  Fold 0 Epoch 06 | train_acc=0.991 | val_acc=1.000
  Fold 0 Epoch 07 | train_acc=1.000 | val_acc=1.000
  Fold 0 Epoch 08 | train_acc=0.991 | val_acc=1.000
  Fold 0 Epoch 09 | train_acc=0.991 | val_acc=1.000
  Fold 0 Epoch 10 | train_acc=0.974 | val_acc=0.967
  Fold 0 Epoch 11 | train_acc=1.000 | val_acc=1.000
  Fold 0 Epoch 12 | train_acc=1.000 | val_acc=0.967
  Fold 0 Epoch 13 | train_acc=0.991 | val_acc=0.967
  Fold 0 Epoch 14 | train_acc=1.000 | val_acc=1.000
  Fold 0 Epoch 15 | train_acc=1.000 | val_acc=1.000
  Fold 0 Epoch 16 | train_acc=1.000 | val_acc=0.967
  Fold 0 Epoch 17 | train_acc=1.000 | val_acc=1.000
  Fold 0 Epoch 18 | train_acc=1.000 | val_acc=1.000

In [24]:
from sklearn.model_selection import StratifiedKFold
import numpy as np
import torch
from torch.utils.data import DataLoader

input_size = X_temp.shape[2]
num_classes = 3
EPOCHS = 20          # จะใช้เท่าของเดิมก่อน
BATCH_SIZE = 16
LR = 1e-3            # fix lr ไว้ก่อน ค่อยเล่นทีหลังได้

# กำหนดชุด hyperparameters ที่จะลอง
hyperparam_configs = [
    {"name": "h64_l1",  "hidden_size": 64,  "num_layers": 1},
    {"name": "h64_l2",  "hidden_size": 64,  "num_layers": 2},
    {"name": "h128_l1", "hidden_size": 128, "num_layers": 1},
    {"name": "h128_l2", "hidden_size": 128, "num_layers": 2},
]

In [25]:
y_temp_np = y_temp.cpu().numpy() if isinstance(y_temp, torch.Tensor) else np.array(y_temp)

results = []  # เอาไว้เก็บผลลัพธ์ของแต่ละ config

for cfg in hyperparam_configs:
    print("\n==============================")
    print(f"Config: {cfg['name']} | hidden={cfg['hidden_size']} | layers={cfg['num_layers']}")
    print("==============================")

    # สร้าง KFold ใหม่สำหรับ config นี้ (หรือจะใช้ตัวเดิมก็ได้)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    fold_val_accs = []
    fold_idx = 0

    for train_idx, val_idx in kfold.split(np.zeros(len(y_temp_np)), y_temp_np):
        print(f"\n===== Fold {fold_idx} =====")

        X_train_fold = X_temp[train_idx]
        y_train_fold = y_temp[train_idx]
        X_val_fold   = X_temp[val_idx]
        y_val_fold   = y_temp[val_idx]

        print("  Train fold size:", len(X_train_fold))
        print("  Val   fold size:", len(X_val_fold))

        # Dataset / DataLoader
        train_ds = CSIDataset(X_train_fold, y_train_fold)
        val_ds   = CSIDataset(X_val_fold,   y_val_fold)

        train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
        val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False)

        # สร้าง model ใหม่สำหรับ fold นี้
        model = LSTMClassifier(
            input_size=input_size,
            hidden_size=cfg["hidden_size"],
            num_layers=cfg["num_layers"],
            num_classes=num_classes
        ).to(DEVICE)

        criterion = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=LR)

        # ===== Train บน fold นี้ =====
        for epoch in range(1, EPOCHS + 1):
            model.train()
            correct = 0
            total = 0

            for xb, yb in train_loader:
                xb = xb.to(DEVICE)
                yb = yb.to(DEVICE)

                optimizer.zero_grad()
                logits = model(xb)
                loss = criterion(logits, yb)
                loss.backward()
                optimizer.step()

                preds = logits.argmax(dim=1)
                correct += (preds == yb).sum().item()
                total += yb.size(0)

            train_acc = correct / total

            # วัด val accuracy
            model.eval()
            val_correct = 0
            val_total = 0

            with torch.no_grad():
                for xb, yb in val_loader:
                    xb = xb.to(DEVICE)
                    logits = model(xb)
                    preds = logits.argmax(dim=1).cpu()
                    val_correct += (preds == yb).sum().item()
                    val_total += yb.size(0)

            val_acc = val_correct / val_total

            print(f"  Fold {fold_idx} Epoch {epoch:02d} | train_acc={train_acc:.3f} | val_acc={val_acc:.3f}")

        # ใช้ val_acc ของ epoch สุดท้ายเป็นตัวแทน fold นี้ (เวอร์ชันง่าย)
        fold_val_accs.append(val_acc)
        fold_idx += 1

    mean_val = float(np.mean(fold_val_accs))
    std_val  = float(np.std(fold_val_accs))
    print("\n>>> Config", cfg["name"], "fold val accs:", [round(a, 3) for a in fold_val_accs])
    print(">>> Mean val accuracy:", round(mean_val, 4), "| Std:", round(std_val, 4))

    results.append({
        "name": cfg["name"],
        "hidden_size": cfg["hidden_size"],
        "num_layers": cfg["num_layers"],
        "mean_val_acc": mean_val,
        "std_val_acc": std_val,
    })

print("\n================ FINAL SUMMARY ================\n")
for r in results:
    print(f"{r['name']}: hidden={r['hidden_size']}, layers={r['num_layers']}, "
          f"mean_val_acc={r['mean_val_acc']:.4f}, std={r['std_val_acc']:.4f}")


Config: h64_l1 | hidden=64 | layers=1

===== Fold 0 =====
  Train fold size: 117
  Val   fold size: 30
  Fold 0 Epoch 01 | train_acc=0.692 | val_acc=0.867
  Fold 0 Epoch 02 | train_acc=0.863 | val_acc=0.867
  Fold 0 Epoch 03 | train_acc=0.812 | val_acc=0.833
  Fold 0 Epoch 04 | train_acc=0.855 | val_acc=0.867
  Fold 0 Epoch 05 | train_acc=0.855 | val_acc=0.833
  Fold 0 Epoch 06 | train_acc=0.863 | val_acc=0.833
  Fold 0 Epoch 07 | train_acc=0.897 | val_acc=0.967
  Fold 0 Epoch 08 | train_acc=0.966 | val_acc=0.967
  Fold 0 Epoch 09 | train_acc=0.983 | val_acc=1.000
  Fold 0 Epoch 10 | train_acc=1.000 | val_acc=0.967
  Fold 0 Epoch 11 | train_acc=0.991 | val_acc=1.000
  Fold 0 Epoch 12 | train_acc=1.000 | val_acc=1.000
  Fold 0 Epoch 13 | train_acc=0.991 | val_acc=1.000
  Fold 0 Epoch 14 | train_acc=0.991 | val_acc=1.000
  Fold 0 Epoch 15 | train_acc=1.000 | val_acc=1.000
  Fold 0 Epoch 16 | train_acc=0.991 | val_acc=1.000
  Fold 0 Epoch 17 | train_acc=1.000 | val_acc=1.000
  Fold 0 Epo

In [28]:
import torch

# สมมติ X_train, y_train, X_val, y_val เป็น torch.Tensor อยู่แล้ว
X_train_full = X_temp
y_train_full = y_temp

print("Train_full:", X_train_full.shape)
print("Test      :", X_test.shape)

Train_full: torch.Size([147, 32, 53])
Test      : torch.Size([26, 32, 53])


In [29]:
BATCH_SIZE = 16

train_ds_full = CSIDataset(X_train_full, y_train_full)
test_ds       = CSIDataset(X_test,       y_test)

train_loader_full = DataLoader(train_ds_full, batch_size=BATCH_SIZE, shuffle=True)
test_loader       = DataLoader(test_ds,       batch_size=BATCH_SIZE, shuffle=False)

In [30]:
input_size  = X_train_full.shape[2]   # น่าจะ 53
num_classes = 3

HIDDEN_SIZE = 64
NUM_LAYERS  = 1
LR          = 1e-3
EPOCHS      = 30

print("Final config:")
print(f"  input_size  = {input_size}")
print(f"  hidden_size = {HIDDEN_SIZE}")
print(f"  num_layers  = {NUM_LAYERS}")
print(f"  lr          = {LR}")
print(f"  epochs      = {EPOCHS}")

model = LSTMClassifier(
    input_size=input_size,
    hidden_size=HIDDEN_SIZE,
    num_layers=NUM_LAYERS,
    num_classes=num_classes
).to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

Final config:
  input_size  = 53
  hidden_size = 64
  num_layers  = 1
  lr          = 0.001
  epochs      = 30


In [31]:
for epoch in range(1, EPOCHS + 1):
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0

    for xb, yb in train_loader_full:
        xb = xb.to(DEVICE)
        yb = yb.to(DEVICE)

        optimizer.zero_grad()
        logits = model(xb)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * yb.size(0)
        preds = logits.argmax(dim=1)
        correct += (preds == yb).sum().item()
        total += yb.size(0)

    epoch_loss = total_loss / total
    epoch_acc  = correct / total
    print(f"Epoch {epoch:02d} | loss={epoch_loss:.4f} | train_acc={epoch_acc:.4f}")

print("=== Final training finished ===")

Epoch 01 | loss=0.9197 | train_acc=0.5850
Epoch 02 | loss=0.6119 | train_acc=0.8503
Epoch 03 | loss=0.4725 | train_acc=0.9116
Epoch 04 | loss=0.3878 | train_acc=0.9864
Epoch 05 | loss=0.3057 | train_acc=0.9864
Epoch 06 | loss=0.2627 | train_acc=0.9864
Epoch 07 | loss=0.2370 | train_acc=0.9864
Epoch 08 | loss=0.2139 | train_acc=0.9932
Epoch 09 | loss=0.1994 | train_acc=0.9796
Epoch 10 | loss=0.2325 | train_acc=0.9728
Epoch 11 | loss=0.1993 | train_acc=0.9864
Epoch 12 | loss=0.1865 | train_acc=1.0000
Epoch 13 | loss=0.1589 | train_acc=0.9932
Epoch 14 | loss=0.1464 | train_acc=0.9864
Epoch 15 | loss=0.1236 | train_acc=1.0000
Epoch 16 | loss=0.1147 | train_acc=0.9864
Epoch 17 | loss=0.1070 | train_acc=1.0000
Epoch 18 | loss=0.0951 | train_acc=1.0000
Epoch 19 | loss=0.0851 | train_acc=1.0000
Epoch 20 | loss=0.0703 | train_acc=1.0000
Epoch 21 | loss=0.1002 | train_acc=0.9932
Epoch 22 | loss=0.0847 | train_acc=1.0000
Epoch 23 | loss=0.0654 | train_acc=1.0000
Epoch 24 | loss=0.0619 | train_acc

In [32]:
class_names = ["no_human", "static", "movement"]

torch.save({
    "model_state_dict": model.state_dict(),
    "input_size": input_size,
    "hidden_size": HIDDEN_SIZE,
    "num_layers": NUM_LAYERS,
    "num_classes": num_classes,
    "class_names": class_names
}, "csi_lstm_final_70_15_15.pt")

print("✅ Saved final model to csi_lstm_final_70_15_15.pt")

✅ Saved final model to csi_lstm_final_70_15_15.pt


In [33]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

model.eval()
all_preds = []
all_true  = []

with torch.no_grad():
    for xb, yb in test_loader:
        xb = xb.to(DEVICE)
        logits = model(xb)
        preds = logits.argmax(dim=1).cpu().numpy()

        all_preds.extend(preds)
        all_true.extend(yb.numpy())

all_preds = np.array(all_preds)
all_true  = np.array(all_true)

test_acc = (all_preds == all_true).mean()
print(f"\n🎯 Final TEST accuracy: {test_acc:.4f}")

cm = confusion_matrix(all_true, all_preds)
print("\nConfusion Matrix:\n", cm)

print("\nClassification Report:\n")
print(classification_report(all_true, all_preds, target_names=class_names))


🎯 Final TEST accuracy: 1.0000

Confusion Matrix:
 [[ 3  0  0]
 [ 0 12  0]
 [ 0  0 11]]

Classification Report:

              precision    recall  f1-score   support

    no_human       1.00      1.00      1.00         3
      static       1.00      1.00      1.00        12
    movement       1.00      1.00      1.00        11

    accuracy                           1.00        26
   macro avg       1.00      1.00      1.00        26
weighted avg       1.00      1.00      1.00        26

